# Monte Carlo Radiative Transfer: Single Packet Simulation

Welcome to the first Monte Carlo radiative transfer notebook! In this session, we'll build the fundamental components needed for simulating photon propagation in supernova ejecta.

## What We'll Learn

By the end of this notebook, you'll understand and implement the core processes that make Monte Carlo radiative transfer work:

### Key Functions We'll Work With:

1. **`calculate_distance_to_boundary()`** - *Provided with explanation*
   - Calculate distances to spherical shell boundaries
   - Determine if photon hits inner or outer boundary

2. **`calculate_doppler_factor()`** - *Implementation required*
   - Handle Doppler shifts in expanding supernova ejecta
   - Transform between lab and comoving frames

3. **`calculate_distance_line()`** - *Implementation required*
   - Compute distances to spectral line interactions
   - Account for Doppler effects and resonance conditions

4. **`move_packet()`** - *Provided with explanation*
   - Update photon position using spherical geometry
   - Apply law of cosines for position changes

5. **`scatter_packet()`** - *Provided with explanation*
   - Handle photon scattering physics and frame transformations
   - Conserve energy and apply proper Doppler corrections

6. **`single_packet_loop()`** - *Implementation required*
   - Complete simulation loop combining all components
   - Track photon until escape or capture

### Learning Objectives:

- **Understand spherical geometry** in Monte Carlo simulations
- **Master Doppler physics** in expanding media
- **Implement distance calculations** for boundaries and line interactions
- **Build a complete single-packet simulation** that demonstrates the core physics
- **Connect individual components** to understand how TARDIS works internally

### Teaching Approach:

Some functions will be **provided with detailed explanations** to focus on the physics concepts, while others require **hands-on implementation** to deepen your understanding of the computational methods.

Let's start by setting up our computational environment and defining the supernova parameters!

In [ ]:
import numpy as np

from astropy.visualization import quantity_support
from astropy import units as u, constants as const

quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x10fe381a0>

# Photon Propagation in Spherical Geometry

Up to now, photon positions have been tracked in Cartesian $(x, y)$ coordinates.  
We now switch to **spherical coordinates**, where a packet's position is represented by a radial distance $r$ and a direction angle $\theta$ (measured from the radial axis).

In this formulation:

- The photon travels a distance $s$ in direction $\theta$.
- The updated radius $r_{\text{new}}$ is computed using basic trigonometry or vector methods.
- We only track the radial distance $r$ and direction cosine $\mu = \cos(\theta)$ for simplicity.

## Computational Domain

In TARDIS, the simulation volume is divided into multiple **spherical shells**, known as **cells**. Each cell has:

- An **inner boundary** at radius `R_INNER`
- An **outer boundary** at radius `R_OUTER`

A photon packet (also called a "packet") is injected at the **inner boundary** ($r = \texttt{R\_INNER}$). It **escapes** if it crosses the outer boundary, and is **captured** (terminated) if it crosses the inner boundary again.

In reality, the structure of a supernova is complex: different layers can have different **densities**, **compositions**, and **temperatures**. For computational efficiency, we approximate the structure as a set of cells, each with **uniform properties**. That means each cell has a constant electron number density, opacity, and ionization state.

As a photon moves between cells, the physical properties such as electron number density (relevant for scattering) may change.

For now, we will simplify this further by working with **only one shell**.

---

## $\blacktriangleright$ Setting up the Simulation

In this task, you'll set up the basic parameters for our supernova simulation by calculating the shell radii and defining the spectral line of interest.

### Task 1: Calculate Supernova Shell Radii

Calculate the inner and outer radii of a supernova shell using homologous expansion.

**Physical parameters:**
- **Time since explosion**: 15 days
- **Inner velocity**: 8000 km/s  
- **Outer velocity**: 12000 km/s

**Your task:**
1. Define the time and velocities using astropy.units (you'll need to figure out how to convert the values above into proper astropy quantities)
2. Calculate the radii using homologous expansion: $R = V \times \text{TIME\_EXPLOSION}$
3. Convert the radii to centimeters using the `.to(u.cm)` method

**Remember:** In homologous expansion, material at radius $r$ has velocity $v = r/t$, so $r = v \times t$.

### Task 2: Define the Spectral Line

Define the Si II spectral line at 6373.133 Angstrom (vacuum wavelength) that we'll use for line interactions.

**Your task:**
1. Define `LINE_LAMBDA` to be  $6373.133\, \AA$ (vacuum wavelength)
2. Convert to frequency: `LINE_NU` should be LINE_LAMBDA converted to Hz (using `equivalencies=u.spectral()` to make Astropy understand that you are thinking about wavelength and frequencies of waves)

**Note:** This is the Si II line that appears prominently in Type Ia supernova spectra. Specifically this is one of the lines that is part of the doublet that makes up this this line (the other being $6348.864\,\AA$)

---

<details>
<summary><strong>💡 Hint 1</strong></summary>

Use astropy units to define the physical quantities:
```python
TIME_EXPLOSION = 15 * u.day
V_INNER = 8000 * u.km / u.s
LINE_LAMBDA = 6373.133 * u.Angstrom
```
</details>

<details>
<summary><strong>💡 Hint 2</strong></summary>

The radii should be very large numbers when converted to cm. If you get small values, check your unit conversions.
</details>

In [6]:
# Your code goes here
# Task 1: Define the time since explosion and velocities
# Then calculate R_INNER and R_OUTER using homologous expansion

# Step 1: Define time and velocities with proper units
# TIME_EXPLOSION = ? (15 days)
# V_INNER = ? (8000 km/s)
# V_OUTER = ? (12000 km/s)

# Step 2: Calculate radii using R = V * TIME
# R_INNER = ?
# R_OUTER = ?

# Step 3: Convert to centimeters
# R_INNER = ?
# R_OUTER = ?

# Task 2: Define the spectral line
# Step 1: Define the Si II line wavelength (vacuum)
# LINE_LAMBDA = ? (6373.133 Angstrom)

# Step 2: Convert to frequency
# LINE_NU = ?

In [ ]:
# SOLUTION
# Task 1: Calculate supernova shell radii
# Step 1: Define time and velocities
TIME_EXPLOSION = 15 * u.day
V_INNER = 8000 * u.km / u.s
V_OUTER = 12000 * u.km / u.s

# Step 2: Calculate radii using homologous expansion R = V * t
R_INNER = (V_INNER * TIME_EXPLOSION).to(u.cm)
R_OUTER = (V_OUTER * TIME_EXPLOSION).to(u.cm)
\
# Task 2: Define the spectral line
# Step 1: Define the Si II line wavelength (vacuum)
LINE_LAMBDA = 6373.133 * u.Angstrom

# Step 2: Convert to frequency
LINE_NU = LINE_LAMBDA.to(u.Hz, equivalencies=u.spectral())


Inner radius: 1.04e+15 cm
Outer radius: 1.56e+15 cm
Line wavelength: 6373.133 Angstrom
Line frequency: 4.70e+14 Hz


## Calculating the Distance to the Next Shell Boundary - **Explanation**

Your goal is to understand how far a photon must travel before it reaches the next boundary of the current spherical shell, and whether that boundary is the **inner** or **outer** shell.

The mathematics for this calculation comes from solving the intersection of a ray with spherical surfaces. The geometry depends on the direction of photon motion:

### Case 1: Outward Motion ($\mu > 0$)

When a photon moves outward, it will always hit the outer boundary:

![Distance to outer boundary](https://tardis-sn.github.io/tardis/_images/d_outer.png)

The distance calculation uses the quadratic formula from the intersection of the ray with the outer sphere:

$$d_{\text{outer}} = -r\mu + \sqrt{r_{\text{outer}}^2 + (\mu^2 - 1)r^2}$$

### Case 2: Inward Motion ($\mu < 0$) 

When a photon moves inward, it may hit either the inner boundary or miss it and continue to the outer boundary:

![Distance to inner boundary](https://tardis-sn.github.io/tardis/_images/d_inner.png)

For the inner boundary to be reachable, the discriminant must be non-negative:
$$r_{\text{inner}}^2 + (\mu^2 - 1)r^2 \geq 0$$

If reachable, the distance to the inner boundary is:
$$d_{\text{inner}} = -r\mu - \sqrt{r_{\text{inner}}^2 + (\mu^2 - 1)r^2}$$

If not reachable (discriminant < 0), the photon will hit the outer boundary instead.

### Algorithm Logic

The implementation follows this decision tree:

1. **If $\mu > 0$**: Calculate distance to outer boundary (always reachable)
2. **If $\mu < 0$**: 
   - Check if inner boundary is reachable
   - If yes: Calculate distance to inner boundary
   - If no: Calculate distance to outer boundary (packet misses inner boundary)

### Function Provided

The `calculate_distance_to_boundary()` function implements this logic with proper handling of edge cases.

Refer to the TARDIS documentation for the complete mathematical derivation:  
https://tardis-sn.github.io/tardis/physics/montecarlo/propagation.html#distance-to-next-cell

In [8]:
# PROVIDED IMPLEMENTATION - Execute cell
def calculate_distance_to_boundary(r, mu, r_inner, r_outer):
    """
    Calculate the distance to the next shell boundary and which boundary is hit.

    Parameters
    ----------
    r : astropy Quantity
        Current radial position
    mu : float
        Direction cosine (cos(theta))
    r_inner : astropy Quantity
        Inner boundary radius
    r_outer : astropy Quantity
        Outer boundary radius

    Returns
    -------
    distance : astropy Quantity
        Distance to the next boundary
    is_outer_boundary : bool
        True if outer boundary is hit first, False if inner boundary is hit
    """
    if mu > 0:
        # moving outward
        term = r_outer**2 + (mu**2 - 1.0) * r**2
        distance = -r * mu + np.sqrt(term)
        return distance, True
    else:
        # moving inward
        check = r_inner**2 + (mu**2 - 1.0) * r**2
        if check >= 0.0:
            distance = -r * mu - np.sqrt(check)
            return distance, False
        else:
            # misses inner boundary, will hit outer boundary
            term = r_outer**2 + (mu**2 - 1.0) * r**2
            distance = -r * mu + np.sqrt(term)
            return distance, True

# Doppler Effects in Expanding Supernovae

In addition to calculating distances to shell boundaries, photons in supernovae can interact with **spectral lines** in the expanding ejecta. This introduces new physics related to the **Doppler effect** and **frame transformations**.

## Key Concepts

1. **Expanding Medium**: In supernovae, the ejecta is expanding with velocity $v = r/t$ (homologous expansion)
2. **Doppler Shifts**: As photon packets move through the expanding medium, their frequencies change
3. **Resonance Condition**: A photon packet interacts with a line when its comoving frame frequency matches the line rest frequency
4. **Comoving vs Lab Frame**: We need to transform between the observer's frame (lab) and the local rest frame of the expanding material (comoving)

## Physics of Doppler Transformations

The Doppler factor relates frequencies in different frames:
$$\text{Doppler factor} = (1 - \mu \beta)$$

where:
- $\beta = v/c$ is the velocity in units of the speed of light  
- $\mu = \cos(\theta)$ is the direction cosine
- $v = r/t$ is the local expansion velocity

**Frequency transformations:**
- Rest $\rightarrow$ Comoving: $\nu_{\text{packet,comoving}} = \nu_{\text{packet,rest}} \times (1 - \mu\beta)$  
- Comoving $\rightarrow$ Rest: $\nu_{\text{packet,rest}} = \nu_{\text{packet,comoving}} / (1 - \mu\beta)$

## $\blacktriangleright$ Task: Doppler Transformation Function

Let's create a function to calculate Doppler shifts and explore different photon trajectories in an expanding supernova.

**Your task:** Write a function that calculates the Doppler factor for a photon packet.

**Implementation steps:**
1. Calculate expansion velocity: $v = r / t_{\text{explosion}}$
2. Calculate dimensionless velocity: $\beta = v / c$
3. Calculate Doppler factor: $\text{Doppler factor} = (1 - \mu \beta)$
4. Return the Doppler factor

**Then test your function** with a specific photon packet scenario:

**Photon Packet Setup:**
- **Packet wavelength**: 6370 Å (slightly blue-shifted from Si II line at 6371.36 Å)
- **Packet rest frequency**: Calculate from wavelength using the spectral equivalency

**Test scenarios** to see how the photon packet comoving frame frequency changes:
- **Scenario A**: Packet moving radially outward (`mu = +1.0`) at outer boundary
- **Scenario B**: Packet moving radially inward (`mu = -1.0`) at outer boundary  
- **Scenario C**: Packet moving tangentially (`mu = 0.0`) at middle of shell
- **Scenario D**: Packet moving outward at 45° (`mu = 0.7`) at inner boundary

**Physics questions to explore:**
- Which scenario gives the strongest blue-shift? Red-shift?
- Why does the direction matter for the Doppler effect?
- How does position in the ejecta affect the comoving frame frequency shift?

---

<details>
<summary><strong>💡 Hint 1</strong></summary>

Remember to use `.to(1)` to make beta dimensionless:
```python
beta = (v / const.c).to(1)
```
</details>

<details>
<summary><strong>💡 Hint 2</strong></summary>

The Doppler factor should be close to 1.0 for typical supernova velocities (~0.1c). If you get very large or very small values, check your velocity calculation.
</details>

In [9]:
# Task: Implement Doppler transformation function
# Your code goes here

def calculate_doppler_factor(r, mu, time_explosion):
    """
    Calculate the Doppler factor for a photon packet in expanding ejecta.
    
    Parameters
    ----------
    r : astropy Quantity
        Current radial position
    mu : float
        Direction cosine (positive = outward, negative = inward)
    time_explosion : astropy Quantity
        Time since explosion
        
    Returns
    -------
    doppler_factor : float
        Doppler factor (dimensionless)
    """
    # Step 1: Calculate expansion velocity: v = r / time_explosion
    
    # Step 2: Calculate dimensionless velocity: beta = v / c
    # (Hint: use const.c and .to(1) to make dimensionless)
    
    # Step 3: Calculate Doppler factor: (1 - mu * beta)
    
    return 1.0  # placeholder

In [10]:
# SOLUTION: Doppler transformation function

def calculate_doppler_factor(r, mu, time_explosion):
    """
    Calculate the Doppler factor for a photon packet in expanding ejecta.
    
    Parameters
    ----------
    r : astropy Quantity
        Current radial position
    mu : float  
        Direction cosine (positive = outward, negative = inward)
    time_explosion : astropy Quantity
        Time since explosion
        
    Returns
    -------
    doppler_factor : float
        Doppler factor (dimensionless)
    """
    # Step 1: Calculate expansion velocity
    v = r / time_explosion
    
    # Step 2: Calculate dimensionless velocity 
    beta = (v / const.c).to(1)
    
    # Step 3: Calculate Doppler factor
    doppler_factor = (1 - mu * beta)  # ensure dimensionless
    
    return doppler_factor

## Distance to Line Interactions

Now that we understand Doppler transformations, we can calculate how far an MC packet must travel before it reaches resonance with a spectral line.

### The Physics

An MC packet will interact with a line when its comoving frequency equals the line frequency. Since the comoving frequency changes as the MC packet moves through the expanding medium (due to changing velocity), we need to calculate the distance to resonance.

### Distance Calculation

The calculation involves finding where along the packet's trajectory the Doppler-shifted frequency matches the line rest frequency. In the expanding ejecta, the velocity changes with radius according to homologous expansion ($v = r/t$), which means the Doppler shift changes continuously as the packet moves.

For a packet with rest frequency $\nu_{\text{lab}}$ moving in direction $\mu$ at radius $r$, the comoving frequency is:

$$\nu_{\text{comoving}} = \nu_{\text{lab}} \times (1 - \mu\beta)$$

where $\beta = v/c = r/(ct)$ is the dimensionless velocity.

As the packet moves distance $d$ and reaches a new radius $r'$, the new comoving frequency becomes $\nu_{\text{lab}} \times (1 - \mu\beta')$ where $\beta' = r'/(ct)$.

The distance to line interaction is given by:

$$d_{\text{line}} = \frac{(\nu_{\text{comoving}} - \nu_{\text{line}})}{\nu_{\text{lab}}} \times c \times t$$

where:
- $\nu_{\text{comoving}}$ is the MC packet frequency in the comoving frame at current position
- $\nu_{\text{line}}$ is the rest frequency of the spectral line
- $\nu_{\text{lab}}$ is the MC packet frequency in the lab frame
- $c$ is the speed of light
- $t$ is the time since explosion

### Key Points

- **If $\nu_{\text{comoving}} > \nu_{\text{line}}$**: MC packet is blue-shifted relative to line, will reach resonance
- **If $\nu_{\text{comoving}} \leq \nu_{\text{line}}$**: MC packet cannot reach resonance (return very large distance)
- **Distance depends on**: Current position, direction, MC packet frequency, and expansion time

A more complete description is available in the [TARDIS documentation](https://tardis-sn.github.io/tardis/physics/montecarlo/propagation.html#propagation-in-a-spherical-domain).

### $\blacktriangleright$ Task 2: Implement Line Distance Calculation

Now you'll implement the function that calculates how far a photon packet must travel before it reaches resonance with a spectral line.

**Physics:** The distance to line interaction is given by:

$$d_{\text{line}} = \frac{(\nu_{\text{packet,comoving}} - \nu_{\text{line,rest}})}{\nu_{\text{packet,rest}}} \times c \times t$$

where $\nu_{\text{packet,comoving}} = \nu_{\text{packet,rest}} \times (1 - \mu\beta)$ and $\beta = v/c = r/(ct)$.

**Test your function** with several scenarios:

1. **Blue packet**: Use a packet frequency much bluer than the line rest frequency (e.g., from ~6000-6300 Å). Remember that lab frequency is "defined"" at the center. Thus they are redshifted at the inner boundary at radius `R_INNER`. A packet needs to be significantly bluer than the line frequency to still have enough "blue-ness" left to reach resonance as it travels outward through the expanding ejecta. (use the doppler-factor to help you).

2. **Red packet**: Use a redder frequency than the line rest frequency - should return large distance since it can never reach resonance.

3. **Critical test**: Use the exact frequency that would red-shift to line rest frequency at the outer boundary.

**Physical Insight:** In expanding supernovae, packets are continuously red-shifted as they move outward. Only packets that start sufficiently blue-shifted (much bluer than the line frequency) can eventually reach resonance with the line somewhere in their outward journey. Packets that are only slightly bluer than the line frequency may have already been red-shifted past resonance by the time they reach the inner boundary.

---

<details>
<summary><strong>💡 Hint 1</strong></summary>

Use your Doppler factor function and check the resonance condition:
```python
if comov_nu > nu_line:
    # Calculate distance
else:
    return 1e99 * u.cm  # Very large distance
```
</details>

<details>
<summary><strong>💡 Hint 2</strong></summary>

The distance should be in cm units. Blue packets should give reasonable distances, while red packets should return the large fallback value.
</details>

In [13]:
# Task 2: Implement line distance calculation
# Your code goes here

def calculate_distance_line(r, mu, nu_rest, nu_line, time_explosion):
    """
    Calculate distance until photon packet is in resonance with a spectral line.
    
    Parameters
    ----------
    r : astropy Quantity
        current radial position
    mu : float
        direction cosine (dimensionless)
    nu_rest : astropy Quantity
        photon packet rest frequency
    nu_line : astropy Quantity
        line rest frequency
    time_explosion : astropy Quantity
        time since explosion
    
    Returns
    -------
    distance : astropy Quantity
        distance to line interaction, or very large number if no interaction
    """
    # Step 1: Calculate Doppler factor using the function you just implemented
    
    # Step 2: Transform packet frequency to comoving frame
    
    # Step 3: Check if resonance is possible (comoving frequency > line frequency)
    # If yes: calculate distance = (freq_diff / nu_rest) * c * time_explosion
    # If no: return very large distance (1e99 * u.cm)
    
    # Remember to use .to(u.cm) to ensure proper units on return
    
    distance = 1e99 * u.cm  # placeholder
    return distance.to(u.cm)

In [14]:
# SOLUTION: Line distance calculation

def calculate_distance_line(r, mu, nu_rest, nu_line, time_explosion):
    """
    Calculate distance until photon packet is in resonance with a spectral line.
    
    Parameters
    ----------
    r : astropy Quantity
        current radial position
    mu : float
        direction cosine (dimensionless)
    nu_rest : astropy Quantity
        photon packet rest frequency
    nu_line : astropy Quantity
        line rest frequency
    time_explosion : astropy Quantity
        time since explosion
    
    Returns
    -------
    distance : astropy Quantity
        distance to line interaction, or very large number if no interaction
    """
    # Step 1: Calculate Doppler factor using existing function
    doppler_factor = calculate_doppler_factor(r, mu, time_explosion)
    
    # Step 2: Transform to comoving frame
    comov_nu = nu_rest * doppler_factor
    
    # Step 3: Check if resonance is possible and calculate distance
    if comov_nu > nu_line:
        distance = (comov_nu - nu_line) / nu_rest * const.c * time_explosion
        return distance.to(u.cm)
    else:
        return (1e99 * u.cm)  # Very large distance - no interaction

In [ ]:
# SOLUTION: Testing the line distance calculation function

# Test the calculate_distance_line function with different scenarios
print("Testing calculate_distance_line function:")

# Test 1: Blue packet (properly blue-shifted accounting for Doppler effect at R_INNER)
# Calculate how blue the packet needs to be to still reach resonance after red-shifting from center to R_INNER
doppler_factor_inner = calculate_doppler_factor(R_INNER, 0.8, TIME_EXPLOSION)
blue_nu = LINE_NU * 1.001 / doppler_factor_inner  # Account for red-shift from center to R_INNER
distance_blue = calculate_distance_line(R_INNER, 0.8, blue_nu, LINE_NU, TIME_EXPLOSION)
print(f"\nTest 1 - Blue packet (accounting for Doppler shift at R_INNER):")
print(f"  Doppler factor at R_INNER: {doppler_factor_inner:.3f}")
print(f"  Packet frequency: {blue_nu:.2e}")
print(f"  Distance to interaction: {distance_blue:.2e}")

# Test 2: Very blue packet (should reach resonance quickly)  
very_blue_nu = LINE_NU * 1.15 / doppler_factor_inner  # Also account for Doppler effect
distance_very_blue = calculate_distance_line(R_INNER, 0.6, very_blue_nu, LINE_NU, TIME_EXPLOSION)
print(f"\nTest 2 - Very blue packet (15% bluer):")
print(f"  Packet frequency: {very_blue_nu:.2e}")
print(f"  Distance to interaction: {distance_very_blue:.2e}")

# Test 3: Red packet (should return large distance - no interaction)
red_nu = LINE_NU * 0.95  # 5% redder than line frequency
distance_red = calculate_distance_line(R_INNER, 0.8, red_nu, LINE_NU, TIME_EXPLOSION)
print(f"\nTest 3 - Red packet (5% redder):")
print(f"  Packet frequency: {red_nu:.2e}")
print(f"  Distance to interaction: {distance_red:.2e}")

# Test 4: Check Doppler effect at different positions
print(f"\nTest 4 - Same blue packet at different positions:")
for r_test in [R_INNER, (R_INNER + R_OUTER)/2, R_OUTER * 0.9]:
    distance_test = calculate_distance_line(r_test, 0.8, blue_nu, LINE_NU, TIME_EXPLOSION)
    print(f"  At r = {r_test:.2e}: distance = {distance_test:.2e}")

print(f"\nPhysics insight: Blue packets can reach resonance, red packets cannot.")
print(f"Distance decreases at larger radii due to lower expansion velocities.")

Testing calculate_distance_line function:

Test 1 - Blue packet (accounting for Doppler shift at R_INNER):
  Doppler factor at R_INNER: 0.979
  Packet frequency: 5.05e+14 Hz
  Distance to interaction: 1.81e+15 cm

Test 2 - Very blue packet (15% bluer):
  Packet frequency: 5.53e+14 Hz
  Distance to interaction: 5.17e+15 cm

Test 3 - Red packet (5% redder):
  Packet frequency: 4.47e+14 Hz
  Distance to interaction: 1.00e+99 cm

Test 4 - Same blue packet at different positions:
  At r = 1.04e+15 cm: distance = 1.81e+15 cm
  At r = 1.30e+15 cm: distance = 1.60e+15 cm
  At r = 1.40e+15 cm: distance = 1.52e+15 cm

Physics insight: Blue packets can reach resonance, red packets cannot.
Distance decreases at larger radii due to lower expansion velocities.


### Packet Movement in Spherical Geometry - **Explanation**

Before building the complete simulation, we need to understand how packets move through space using spherical geometry.

**Physics:** When a packet moves distance $d$ in direction $\mu$, the new position is calculated using the law of cosines in spherical coordinates:

$$r_{\text{new}} = \sqrt{r^2 + d^2 + 2rd\mu}$$

$$\mu_{\text{new}} = \frac{\mu r + d}{r_{\text{new}}}$$

where:
- $r$ is the current radial position
- $d$ is the distance traveled
- $\mu = \cos(\theta)$ is the direction cosine
- $r_{\text{new}}$ and $\mu_{\text{new}}$ are the updated position and direction

### Geometric Derivation

These formulas come from applying the law of cosines to the triangle formed by:
- The origin (center of supernova)
- The packet's initial position at radius $r$
- The packet's final position at radius $r_{\text{new}}$

The angle between the initial position vector and the direction of motion is $\theta$, where $\mu = \cos(\theta)$.

### Special Cases

The formulas handle all motion directions correctly:

1. **Radial outward** ($\mu = 1.0$): $r_{\text{new}} = r + d$ and $\mu_{\text{new}} = 1.0$
2. **Radial inward** ($\mu = -1.0$): $r_{\text{new}} = |r - d|$ and $\mu_{\text{new}} = -1.0$
3. **Angled motion** ($-1 < \mu < 1$): Intermediate values following spherical trigonometry

### Function Provided

The `move_packet()` function implements this spherical geometry correctly for all angles and distances.

In [ ]:
# PROVIDED IMPLEMENTATION
def move_packet(r, mu, distance):
    """
    Move a packet through spherical geometry.
    
    Parameters
    ----------
    r : float or astropy Quantity
        Current radial position
    mu : float
        Current direction cosine
    distance : float or astropy Quantity (same units as r)
        Distance to move
        
    Returns
    -------
    new_r : float or astropy Quantity
        New radial position
    new_mu : float
        New direction cosine
    """
    # Step 1: Calculate new radius using law of cosines
    new_r = np.sqrt(r**2 + distance**2 + 2*r*distance*mu)
    
    # Step 2: Calculate new direction cosine
    new_mu = (mu*r + distance) / new_r
    
    return u.Quantity(new_r, u.cm), u.Quantity(new_mu, 1)

### $\blacktriangleright$ Task 4: Implement Packet Scattering Function

Before building the complete simulation, we need to understand how packet scattering works when a photon interacts with a spectral line.

**Physics:** When a packet reaches resonance with a line, the scattering process involves proper frame transformations:

**Important:** We implicitly transform the packet frequency to the comoving frame, where it becomes exactly the line rest frequency at the moment of scattering.

1. **Energy Conservation in Comoving Frame**: Transform packet energy to comoving frame: $E_{\text{comoving}} = E_{\text{lab}} \times (1 - \mu\beta)$
2. **Isotropic Scattering**: Packet scatters isotropically with new random direction: $\mu_{\text{new}} = \text{uniform}(-1, 1)$  
3. **Frequency Reset**: After scattering, packet has line rest frequency in comoving frame ($\nu_{\text{comoving}} = \nu_{\text{line,rest}}$)
4. **Transform Back to Lab Frame**: 
   - Energy: $E_{\text{lab,new}} = E_{\text{comoving}} / (1 - \mu_{\text{new}}\beta)$
   - Frequency: $\nu_{\text{lab,new}} = \nu_{\text{line,rest}} / (1 - \mu_{\text{new}}\beta)$

**Key Physics Insight:** The packet "goes into the atom" (transforms to comoving frame), scatters isotropically, then "comes out of the atom" (transforms back to lab frame) with:
- Conserved energy in the comoving frame
- Line rest frequency in the comoving frame  
- New lab frame properties based on new direction

**Implementation:** Create a function that handles only the scattering transformation (the scattering decision should be handled outside).

---

<details>
<summary><strong>💡 Hint 1</strong></summary>

The scattering sequence is: Lab → Comoving → Scatter $\rightarrow$ Lab. Energy is conserved in the comoving frame:
```python
comov_energy = energy * doppler_factor
new_energy = comov_energy / new_doppler_factor
```
</details>

<details>
<summary><strong>💡 Hint 2</strong></summary>

After scattering, the packet has the line frequency in the comoving frame, which transforms back to lab frame with the new direction.
</details>

In [ ]:
# Task 4: Implement packet scattering function
# Your code goes here

def scatter_packet(r, mu, energy, nu_line, time_explosion):
    """
    Handle packet scattering frame transformations when scattering occurs.
    
    Note: This function assumes scattering WILL occur - the scattering decision
    (optical depth sampling) should be handled outside this function.
    
    Parameters
    ----------
    r : astropy Quantity
        Current radial position
    mu : float
        Current direction cosine  
    energy : float
        Current packet energy in lab frame
    nu_line : astropy Quantity
        Line rest frequency
    time_explosion : astropy Quantity
        Time since explosion
        
    Returns
    -------
    new_mu : float
        New direction cosine after isotropic scattering
    new_energy : float
        New packet energy in lab frame (conserved in comoving frame)
    new_nu : astropy Quantity
        New packet frequency in lab frame (line frequency in comoving frame)
    """
    # Step 1: Calculate current Doppler factor (reuse your function)
    
    # Step 2: Transform energy to comoving frame: energy * doppler_factor
    
    # Step 3: Generate new random direction: np.random.uniform(-1, 1)
    
    # Step 4: Calculate new Doppler factor with new direction
    
    # Step 5: Transform back to lab frame:
    # - new_energy = comoving_energy / new_doppler_factor
    # - new_nu = nu_line / new_doppler_factor
    # - Ensure proper units: use .to() for unit conversion or u.Quantity(value, unit) if needed
    # - new_nu should maintain frequency units (Hz)
    
    return u.Quantity(new_mu, 1), u.Quantity(new_energy, u.erg), u.Quantity(new_nu, u.Hz)

In [ ]:
# SOLUTION: Packet scattering function

def scatter_packet(r, mu, energy, nu_line, time_explosion):
    """
    Handle packet scattering frame transformations when scattering occurs.
    
    Note: This function assumes scattering WILL occur - the scattering decision
    (optical depth sampling) should be handled outside this function.
    
    Parameters
    ----------
    r : astropy Quantity
        Current radial position
    mu : float
        Current direction cosine  
    energy : float
        Current packet energy in lab frame
    nu_line : astropy Quantity
        Line rest frequency
    time_explosion : astropy Quantity
        Time since explosion
        
    Returns
    -------
    new_mu : float
        New direction cosine after isotropic scattering
    new_energy : float
        New packet energy in lab frame (conserved in comoving frame)
    new_nu : astropy Quantity
        New packet frequency in lab frame (line frequency in comoving frame)
    """
    # Step 1: Calculate current expansion velocity and beta
    v = r / time_explosion
    beta = (v / const.c).to(1)  # dimensionless
    
    # Step 2: Transform energy to comoving frame  
    current_doppler_factor = (1 - mu * beta)
    comov_energy = energy * current_doppler_factor
    
    # Step 3: Isotropic scattering - new random direction
    new_mu = np.random.uniform(-1, 1)
    
    # Step 4: Transform back to lab frame with new direction
    new_doppler_factor = (1 - new_mu * beta)
    new_energy = comov_energy / new_doppler_factor  # conserve energy in comoving frame
    new_nu = nu_line / new_doppler_factor  # line frequency in comoving frame, maintains units
    
    return u.Quantity(new_mu, 1), u.Quantity(new_energy, u.erg), u.Quantity(new_nu, u.Hz)

# Single Packet Propagation Simulation

Now that you have all the distance and movement functions, let's build a complete single-packet simulation that handles both boundary crossings and line interactions!

## The Algorithm

For a single MC packet, we repeatedly:
1. **Calculate distances** to both boundary and line interactions using our functions
2. **Choose the closer event** (shorter distance)
3. **Move the packet** to the interaction point using spherical geometry
4. **Handle the interaction**:
   - If boundary hit: packet escapes or is captured (simulation ends)
   - If line hit: check if scattering occurs, update packet properties

## Key Physics

- **Optical depth**: Use $\tau_{\text{event}} = -\ln(\text{random})$ to determine if scattering occurs
- **Sobolev optical depth**: $\tau_{\text{Sobolev}} = 1.0$ (strength of line interaction)  
- **Scattering**: If $\tau_{\text{event}} < \tau_{\text{Sobolev}}$, packet scatters with new random direction
- **Position update**: Use the move_packet function with spherical geometry

## Functions to Use

1. $\text{calculate\_doppler\_factor}(r, \mu, t_{\text{explosion}})$ - for Doppler shifts
2. $\text{calculate\_distance\_to\_boundary}(r, \mu, r_{\text{inner}}, r_{\text{outer}})$ - for boundary crossings  
3. $\text{calculate\_distance\_line}(r, \mu, \nu_{\text{rest}}, \nu_{\text{line}}, t_{\text{explosion}})$ - for line interactions
4. $\text{move\_packet}(r, \mu, d)$ - for position updates
5. $\text{scatter\_packet}(r, \mu, \text{energy}, \nu_{\text{line}}, t_{\text{explosion}})$ - for scattering frame transformations

**Note:** Scattering decision (optical depth sampling) should be handled in the main simulation loop, not inside the scatter function.
As done with the electron scattering this morning.

## $\blacktriangleright$ Task 5: Complete Single Packet Propagation Function

Now implement a clean single-packet propagation function that combines all the physics we've learned.

**Your task:** Create a function `single_packet_loop()` that takes a packet's initial conditions and returns its final state.

**The End Goal:** Every packet simulation will **always terminate** when the packet reaches a boundary. The packet will either:
- **Escape**: Cross the outer boundary (successful escape from supernova)
- **Be captured**: Cross the inner boundary (absorbed back into supernova core)

Line interactions are intermediate events that may scatter the packet and change its trajectory, but the simulation continues until a boundary is reached.

**Algorithm:**
1. Start with initial packet conditions
2. Calculate distances to boundary and line interactions
3. Compare distances:
   - **If boundary distance is shorter**: Determine escape or capture, return final state (simulation ends)
   - **If line distance is shorter**: Move to line interaction point, check if scattering occurs
4. If scattering occurs: update packet properties and continue loop
5. If no scattering: continue loop (packet passes through without interaction)
6. The loop continues until a boundary distance is shorter, terminating the simulation

**Key Physics - Optical Depth Sampling:**

When a packet reaches a line interaction, we need to determine if scattering actually occurs. This is done using **optical depth sampling**:

- **Draw random optical depth**: $\tau_{\text{event}} = -\ln(\text{random})$ where random is uniform(0,1)
- **Compare with line strength**: If $\tau_{\text{event}} < \tau_{\text{Sobolev}}$, scattering occurs

This Monte Carlo sampling correctly reproduces the exponential attenuation of photons through the medium.

**Physics note:** Due to the physics, packets typically experience at most **one** interactions before escaping or being captured, so no complex loop handling is needed.

---

<details>
<summary><strong>💡 Hint 1</strong></summary>

The main loop structure should be:
- Start with a while loop that continues until a boundary is hit
- Calculate both distance to boundary and distance to line interaction
- Move the packet only if distance to line is closer - otherwise the simulation for that packet has ended -> break out of the loop
- Handle the event: for line interactions check scattering and continue loop, for boundary interactions return the final state
</details>

<details>
<summary><strong>💡 Hint 2</strong></summary>

For optical depth sampling, draw a random tau_event = -ln(random) and compare with tau_sobolev to determine if scattering occurs. The boundary interaction should determine whether the packet escaped (outer boundary) or was captured (inner boundary).
</details>

In [ ]:
# Task 5: Single packet propagation function
# Your code goes here

def single_packet_loop(initial_r, initial_mu, initial_nu, initial_energy, 
                          r_inner, r_outer, line_nu, time_explosion, tau_sobolev):
    """
    Propagate a single packet until it escapes or is captured.
    
    Parameters
    ----------
    initial_r : astropy Quantity
        Initial radial position
    initial_mu : float
        Initial direction cosine
    initial_nu : astropy Quantity
        Initial frequency
    initial_energy : float
        Initial energy
    r_inner, r_outer : astropy Quantity
        Shell boundaries
    line_nu : astropy Quantity
        Line rest frequency
    time_explosion : astropy Quantity
        Time since explosion
    tau_sobolev : float
        Sobolev optical depth
        
    Returns
    -------
    final_state : dict
        Dictionary with keys: 'escaped', 'r', 'mu', 'nu', 'energy', 'n_interactions'
    """
    # Step 1: Initialize packet state from initial conditions
    # r, mu, nu, energy = initial values
    # n_interactions = 0
    
    # Step 2: Main loop - while True:
    #   2a: Calculate distance_boundary and distance_line
    #   2b: Compare distances - if boundary is closer: return final_state and exit
    #   2b.1: if boundary is closer use the break statement to exit the loop
    #   2c: If line is closer: move to line, check scattering with optical depth sampling
    #   2d: If scattering occurs: update packet with scatter_packet(), continue loop
    #   2e: If no scattering: continue loop (packet passes through line) 
    #       and start again at 2a from the new position
    
    return {
        'escaped': True,  # True if outer boundary, False if inner
        'r': initial_r,
        'mu': initial_mu, 
        'nu': initial_nu,
        'energy': initial_energy,
        'n_interactions': 0
    }

In [30]:
# SOLUTION: Single packet propagation function

def single_packet_loop(initial_r, initial_mu, initial_nu, initial_energy, 
                          r_inner, r_outer, line_nu, time_explosion, tau_sobolev):
    """
    Propagate a single packet until it escapes or is captured.
    
    Parameters
    ----------
    initial_r : astropy Quantity
        Initial radial position
    initial_mu : float
        Initial direction cosine
    initial_nu : astropy Quantity
        Initial frequency
    initial_energy : float
        Initial energy
    r_inner, r_outer : astropy Quantity
        Shell boundaries
    line_nu : astropy Quantity
        Line rest frequency
    time_explosion : astropy Quantity
        Time since explosion
    tau_sobolev : float
        Sobolev optical depth
        
    Returns
    -------
    final_state : dict
        Dictionary with keys: 'escaped', 'r', 'mu', 'nu', 'energy', 'n_interactions'
    """
    # Initialize packet state
    r, mu, nu, energy = initial_r, initial_mu, initial_nu, initial_energy
    n_interactions = 0
    
    # Propagation loop
    while True:
        # Calculate distances to boundary and line
        distance_boundary, is_outer = calculate_distance_to_boundary(r, mu, r_inner, r_outer)
        distance_line = calculate_distance_line(r, mu, nu, line_nu, time_explosion)
        
        # Choose closer event and move packet
        if distance_line < distance_boundary:
            # Line interaction - move and potentially scatter
            r, mu = move_packet(r, mu, distance_line)
            
            # Check for scattering using tau_sobolev
            z = np.random.random()
            if z < 1 - np.exp(-tau_sobolev):
                # Scattering occurs
                mu, energy, nu = scatter_packet(r, mu, energy, line_nu, time_explosion)
                n_interactions += 1
        else:
            # Boundary hit - move to boundary and return final state
            r, mu = move_packet(r, mu, distance_boundary)
            return {
                'escaped': is_outer,
                'r': r,
                'mu': mu, 
                'nu': nu,
                'energy': energy,
                'n_interactions': n_interactions
            }

## $\blacktriangleright$ Task 6: Test Your Single Packet Function

Now that you have a complete single packet propagation function, let's test it with physically meaningful scenarios to understand the behavior.

**Your task:** Test your single packet function with different packet types to explore the physics.

### Important: Use Random Seeds for Reproducible Testing

Since Monte Carlo simulations involve random processes (scattering directions, optical depth sampling), you should **always use `np.random.seed()`** before each test to ensure reproducible results:

```python
np.random.seed(42)  # Use any integer
result = single_packet_loop(...)
```

**Why use seeds?**
- **Reproducibility**: Same seed = same results, making debugging easier
- **Comparison**: You can compare results with classmates and instructors
- **Systematic testing**: Explore the stochastic nature by changing seeds

### Suggested Test Cases

1. **Escape Test**: Radial outward packet with red frequency (should escape with 0 interactions)
2. **Line Interaction Test**: Blue packet at angle (may scatter before escaping)
3. **Capture Test**: Packet moving inward (should be captured)

**Important for blue packets:** Remember from the earlier Doppler task that packets need to be **significantly bluer** than the line frequency to still reach resonance after being red-shifted from the center to `R_INNER`. Use the test scenarios from Task 2 to pick an appropriately blue-shifted frequency.

**Test Parameters:**
- Start at `R_INNER` 
- Use `tau_sobolev = 1.0`
- Try different combinations of direction (`mu`) and frequency
- Observe how `n_interactions` and `escaped` values change
- **Use different seeds** to explore the statistical nature of the results

Each test reveals different aspects of the physics:
- **Frequency dependence**: Blue vs red packets behave differently
- **Direction dependence**: Radial vs angled trajectories  
- **Line interactions**: How Doppler shifts affect resonance conditions
- **Stochastic behavior**: How randomness affects individual packet paths

In [ ]:
# Task 6: Test your single_packet_loop function
# Your code goes here

# Setup test parameters with clear variable names
DOPPLER_FACTOR_INNER = calculate_doppler_factor(R_INNER, 0.7, TIME_EXPLOSION)
PACKET_INITIAL_NU = LINE_NU * 1.001 / DOPPLER_FACTOR_INNER  # Minimal blue shift
PACKET_INITIAL_MU = 0.7  # Direction cosine (angled motion)
TAU_SOBOLEV = 1.0

# Test: Blue packet at angle
np.random.seed(124)
result = single_packet_loop(
    R_INNER, PACKET_INITIAL_MU, PACKET_INITIAL_NU, 1.0,
    R_INNER, R_OUTER, LINE_NU, TIME_EXPLOSION, TAU_SOBOLEV
)

print(f"Packet frequency: {PACKET_INITIAL_NU:.2e}")
print(f"Escaped: {result['escaped']}, Interactions: {result['n_interactions']}")

result {'escaped': True, 'r': <Quantity 1.5552e+15 cm>, 'mu': <Quantity 0.80304002>, 'nu': <Quantity 4.76809803e+14 Hz>, 'energy': <Quantity 0.99369753>, 'n_interactions': 1}
Packet frequency: 4.80e+14 Hz
Escaped: True, Interactions: 1


In [33]:
# SOLUTION: Complete examples and tests

# Example test cases for the single_packet_loop function
print("Testing single packet propagation with various scenarios:")
print("Note: Using random seeds to make results reproducible for comparison")

# Setup consistent test parameters
DOPPLER_FACTOR_INNER = calculate_doppler_factor(R_INNER, 0.7, TIME_EXPLOSION)
TAU_SOBOLEV = 1.0
print(f"Doppler factor at R_INNER: {DOPPLER_FACTOR_INNER:.3f}")

print("\nTest 1: Red packet moving radially outward (should escape without interactions)")
np.random.seed(42)  # Set seed for reproducible results
PACKET_INITIAL_NU = LINE_NU * 0.99  # Red packet
PACKET_INITIAL_MU = 1.0  # Radial outward
result1 = single_packet_loop(
    R_INNER, PACKET_INITIAL_MU, PACKET_INITIAL_NU, 1.0,  # Red packet, radial outward
    R_INNER, R_OUTER, LINE_NU, TIME_EXPLOSION, TAU_SOBOLEV
)
print(f"  Escaped: {result1['escaped']}, Interactions: {result1['n_interactions']}")

print("\nTest 2: Blue packet at angle (minimal blue shift for line interaction)")
np.random.seed(124)  # Different seed for this test
PACKET_INITIAL_NU = LINE_NU * 1.001 / DOPPLER_FACTOR_INNER  # Minimal blue shift
PACKET_INITIAL_MU = 0.7  # Angled motion
result2 = single_packet_loop(
    R_INNER, PACKET_INITIAL_MU, PACKET_INITIAL_NU, 1.0,  # Blue packet, angled
    R_INNER, R_OUTER, LINE_NU, TIME_EXPLOSION, TAU_SOBOLEV
)
print(f"  Packet frequency: {PACKET_INITIAL_NU:.2e}")
print(f"  Escaped: {result2['escaped']}, Interactions: {result2['n_interactions']}")

print("\nTest 3: Packet moving inward (should be captured)")
np.random.seed(789)  # Seed for this test
PACKET_INITIAL_NU = LINE_NU * 0.99  # Red packet
PACKET_INITIAL_MU = -0.8  # Inward motion
result3 = single_packet_loop(
    R_INNER * 1.1, PACKET_INITIAL_MU, PACKET_INITIAL_NU, 1.0,  # Inward motion
    R_INNER, R_OUTER, LINE_NU, TIME_EXPLOSION, TAU_SOBOLEV
)
print(f"  Escaped: {result3['escaped']}, Interactions: {result3['n_interactions']}")

Testing single packet propagation with various scenarios:
Note: Using random seeds to make results reproducible for comparison
Doppler factor at R_INNER: 0.981

Test 1: Red packet moving radially outward (should escape without interactions)
  Escaped: True, Interactions: 0

Test 2: Blue packet at angle (minimal blue shift for line interaction)
  Packet frequency: 4.80e+14 Hz
  Escaped: True, Interactions: 1

Test 3: Packet moving inward (should be captured)
  Escaped: False, Interactions: 0


# Key Lessons Learned: How TARDIS Really Works

Congratulations! You've now implemented the core physics behind TARDIS's Monte Carlo radiative transfer simulation. Let's summarize the key concepts and see how they connect to the full TARDIS implementation.

## Core Monte Carlo Algorithm

The single-packet simulation you built demonstrates the fundamental algorithm that TARDIS uses for **every energy Monte Carlo packet**:

### 1. **Distance Calculations**
For each packet, TARDIS calculates multiple distances:
- **Distance to boundaries** (inner/outer shell boundaries)
- **Distance to electron scattering** interactions  
- **Distance to line interactions** (for each spectral line in the simulation)

### 2. **Optical Depth Sampling** 
At the start of each packet's journey, TARDIS draws a **random optical depth**:
$$\tau_{\text{event}} = -\ln(\text{random})$$

This single random number determines when the packet will interact during its journey.

### 3. **The Decision Tree**
TARDIS then follows this decision process:

**If the boundary distance is shortest:**
- Move packet to the boundary
- **If inner boundary**: Packet is captured (simulation ends)
- **If outer boundary**: Packet escapes (simulation ends)

**If an interaction distance is shortest:**
- **Do NOT move the packet yet!**
- Instead, check if the interaction WOULD actually occur
- Compare $\tau_{\text{event}}$ with the interaction's optical depth
- **If interaction occurs**: Handle scattering and reset $\tau_{\text{event}}$
- **If interaction doesn't occur**: Subtract the optical depth from $\tau_{\text{event}}$ and continue

### 4. **Cumulative Opacity Effects**
This is the key insight: **Weak interactions add up!**

When a packet "flies through" many weak lines or low-density regions without scattering, TARDIS:
- Subtracts each line's optical depth from $\tau_{\text{event}}$
- Subtracts electron scattering opacity from $\tau_{\text{event}}$
- Eventually, $\tau_{\text{event}}$ becomes so small that even a very weak line interaction (with very small $\tau_{\text{Sobolev}}$) becomes larger than the remaining $\tau_{\text{event}}$, causing scattering to occur

This correctly reproduces the physics: **many weak interactions have the same cumulative effect as one strong interaction**.

## From Simple to Complex

**What you implemented today:**
- Single shell, single line, simplified physics

**Full TARDIS simulation:**
- Multiple shells with different densities and compositions
- Potentially millions of spectral lines
- Both line and electron scattering


But the **core algorithm is identical** to what you built! Every packet follows the same distance calculation → optical depth sampling → decision tree process.

## The Big Picture

You've learned the fundamental physics that makes TARDIS possible:
- **Spherical geometry** for packet movement
- **Doppler physics** in expanding media
- **Frame transformations** for scattering
- **Monte Carlo sampling** for interaction probabilities

These concepts scale directly to full supernova simulations with millions of packets and complex 3D ejecta structures (beyond CURRENT TARDIS capability).

**Well done!** You now understand the heart of how Monte Carlo radiative transfer works in astrophysics. **Next** Let's make this our own small TARDIS

# Summary: Core Monte Carlo Radiative Transfer Implementation

You've successfully implemented the fundamental components of Monte Carlo radiative transfer from first principles. Here are the key computational and physics insights:

## Implementation Highlights

### **Geometric Framework**
- **Spherical coordinates** with homologous expansion: $v = r/t$
- **Ray-sphere intersections** for boundary distance calculations
- **Spherical trigonometry** for packet movement using law of cosines

### **Doppler Physics & Frame Transformations** 
- **Doppler factor**: $(1 - \mu\beta)$ for lab ↔ comoving frame conversions
- **Resonance condition**: Comoving frequency matching line rest frequency
- **Energy conservation** in appropriate reference frames during scattering

### **Monte Carlo Mechanics**
- **Optical depth sampling**: $\tau_{\text{event}} = -\ln(\text{random})$
- **Distance competition**: Boundary vs. line interaction distances
- **Isotropic scattering** with proper frame transformation sequences

## Scaling to Production Codes

**From single packet → TARDIS:**
- Same core algorithm with millions of packets
- Multiple shells, thousands of lines, electron scattering
- Identical distance calculations and decision tree logic

**Key insight**: The fundamental MC algorithm you implemented scales directly to production simulations — TARDIS (or Sedona or ARTIS or SuperNu and others of course) uses the same physics at its core.

**Next**: Combine many packets to generate synthetic spectra and see P Cygni profiles emerge naturally from first principles!

In [1]:
from IPython import get_ipython
from pathlib import Path
ip = get_ipython()
path = None
if '__vsc_ipynb_file__' in ip.user_ns:
    path = ip.user_ns['__vsc_ipynb_file__']
    
nb_path = Path(path)
# Get the current notebook name
current_notebook = nb_path.name

# Create the student version by replacing 'instructor' with 'student'
output_notebook = current_notebook.replace('instructor', 'student')

# Run the nbconvert command
!jupyter nbconvert {current_notebook} --ClearOutputPreprocessor.enabled=True --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['solution']" --to notebook --output {output_notebook}

print(f"Converted {current_notebook} to {output_notebook}")

[NbConvertApp] Converting notebook 3_mc_radiative_transfer_single_packet_instructor.ipynb to notebook
[NbConvertApp] Writing 50562 bytes to 3_mc_radiative_transfer_single_packet_student.ipynb
Converted 3_mc_radiative_transfer_single_packet_instructor.ipynb to 3_mc_radiative_transfer_single_packet_student.ipynb
